In [1]:
import os
import random
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *

from kerashypetune import KerasGridSearch

In [2]:
def set_seed(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

np.random.seed(33)
subset_train = np.random.uniform(0,1, len(y_train))
subset_test = np.random.uniform(0,1, len(y_test))

x_train = x_train[subset_train < 0.1] / 255
y_train = y_train[subset_train < 0.1]

x_test = x_test[subset_test < 0.3] / 255
y_test = y_test[subset_test < 0.3]

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((6036, 28, 28), (6036,), (2963, 28, 28), (2963,))

# Boolean condition

In [4]:
def get_model(param):
    
    set_seed(33)
        
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(param['unit_1'], activation='relu'))
    
    if param['cond']:
        model.add(Dense(32, activation='relu'))
        
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=param['lr']), 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [5]:
param_grid = {
    'unit_1': [128,64], 
    'lr': [1e-2,1e-3], 
    'cond': [True,False],
    'epochs': 100, 
    'batch_size': 256
}

In [6]:
es = EarlyStopping(patience=10, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model

kgs = KerasGridSearch(hypermodel, param_grid, monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
kgs.search(x_train, y_train, validation_data=(x_test, y_test), callbacks=[es])


8 trials detected for ('unit_1', 'lr', 'cond', 'epochs', 'batch_size')

***** (1/8) *****
Search({'unit_1': 128, 'lr': 0.01, 'cond': False, 'epochs': 100, 'batch_size': 256})
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
SCORE: 0.95444 at epoch 15

***** (2/8) *****
Search({'unit_1': 128, 'lr': 0.01, 'cond': True, 'epochs': 100, 'batch_size': 256})
Restoring model weights from the end of the best epoch.
Epoch 00027: early stopping
SCORE: 0.9568 at epoch 18

***** (3/8) *****
Search({'unit_1': 128, 'lr': 0.001, 'cond': False, 'epochs': 100, 'batch_size': 256})
Restoring model weights from the end of the best epoch.
Epoch 00042: early stopping
SCORE: 0.94263 at epoch 32

***** (4/8) *****
Search({'unit_1': 128, 'lr': 0.001, 'cond': True, 'epochs': 100, 'batch_size': 256})
Restoring model weights from the end of the best epoch.
Epoch 00034: early stopping
SCORE: 0.95039 at epoch 27

***** (5/8) *****
Search({'unit_1': 64, 'lr': 0.01, 'cond': False, '

# Variable number of layer

In [7]:
def get_model(param):
    
    set_seed(33)
    
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(param['unit_1'], activation='relu'))
    
    for n in range(param['n_layer'],0,-1):
        model.add(Dense(16*n, activation='relu'))
        
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=param['lr']), 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [8]:
param_grid = {
    'unit_1': [128,64], 
    'lr': [1e-2,1e-3], 
    'n_layer': [4,3],
    'epochs': 100, 
    'batch_size': 256
}

In [9]:
es = EarlyStopping(patience=10, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model

kgs = KerasGridSearch(hypermodel, param_grid, monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
kgs.search(x_train, y_train, validation_data=(x_test, y_test), callbacks=[es])


8 trials detected for ('unit_1', 'lr', 'n_layer', 'epochs', 'batch_size')

***** (1/8) *****
Search({'unit_1': 128, 'lr': 0.01, 'n_layer': 3, 'epochs': 100, 'batch_size': 256})
Restoring model weights from the end of the best epoch.
Epoch 00023: early stopping
SCORE: 0.94668 at epoch 14

***** (2/8) *****
Search({'unit_1': 128, 'lr': 0.01, 'n_layer': 4, 'epochs': 100, 'batch_size': 256})
Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
SCORE: 0.95478 at epoch 14

***** (3/8) *****
Search({'unit_1': 128, 'lr': 0.001, 'n_layer': 3, 'epochs': 100, 'batch_size': 256})
Restoring model weights from the end of the best epoch.
Epoch 00031: early stopping
SCORE: 0.94803 at epoch 30

***** (4/8) *****
Search({'unit_1': 128, 'lr': 0.001, 'n_layer': 4, 'epochs': 100, 'batch_size': 256})
Restoring model weights from the end of the best epoch.
Epoch 00036: early stopping
SCORE: 0.9514 at epoch 26

***** (5/8) *****
Search({'unit_1': 64, 'lr': 0.01, 'n_layer': 3, '

# Conditional layer choice

In [10]:
def get_model(param):
    
    set_seed(33)
    
    layer_choices = {'flat': Flatten(),
                     'pool': GlobalMaxPool2D()}
        
    model = Sequential()
    model.add(Lambda(lambda x: tf.expand_dims(x,-1))) # add a dim for conv2d
    model.add(Conv2D(param['unit'], param['kernel'], activation='relu'))
    model.add(layer_choices[param['layer_types']])
            
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=param['lr']), 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [11]:
param_grid = {
    'unit': [32,64], 
    'kernel' : 3,
    'lr': [1e-2,1e-1], 
    'layer_types': ['pool','flat'],
    'epochs': 100, 
    'batch_size': 512
}

In [12]:
es = EarlyStopping(patience=5, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model

kgs = KerasGridSearch(hypermodel, param_grid, monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
kgs.search(x_train, y_train, validation_data=(x_test, y_test), callbacks=[es])


8 trials detected for ('unit', 'kernel', 'lr', 'layer_types', 'epochs', 'batch_size')

***** (1/8) *****
Search({'unit': 32, 'kernel': 3, 'lr': 0.1, 'layer_types': 'flat', 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
SCORE: 0.92913 at epoch 18

***** (2/8) *****
Search({'unit': 32, 'kernel': 3, 'lr': 0.1, 'layer_types': 'pool', 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
SCORE: 0.108 at epoch 3

***** (3/8) *****
Search({'unit': 32, 'kernel': 3, 'lr': 0.01, 'layer_types': 'flat', 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00013: early stopping
SCORE: 0.96423 at epoch 13

***** (4/8) *****
Search({'unit': 32, 'kernel': 3, 'lr': 0.01, 'layer_types': 'pool', 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00040: early stopping
SCORE: 0.7178

# Conditional learning rate

In [13]:
def get_model(param):
    
    set_seed(33)
    
    opt_choices = {'adam': Adam(),
                   'nadam': Nadam()}
    
    opt = opt_choices[param['opt']]
    opt.lr = param['lr'] 
    
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(param['unit_1'], activation='relu'))
    model.add(Dense(param['unit_2'], activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=opt, 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [14]:
param_grid = {
    'unit_1': [128,64], 
    'unit_2': [64,32],
    'opt': ['adam','nadam'],
    'lr': [1e-2,1e-3], 
    'epochs': 100, 
    'batch_size': 512
}

In [15]:
es = EarlyStopping(patience=5, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model

kgs = KerasGridSearch(hypermodel, param_grid, monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
kgs.search(x_train, y_train, validation_data=(x_test, y_test), callbacks=[es])


16 trials detected for ('unit_1', 'unit_2', 'opt', 'lr', 'epochs', 'batch_size')

***** (1/16) *****
Search({'unit_1': 128, 'unit_2': 64, 'opt': 'adam', 'lr': 0.01, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
SCORE: 0.95174 at epoch 13

***** (2/16) *****
Search({'unit_1': 128, 'unit_2': 64, 'opt': 'adam', 'lr': 0.001, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00033: early stopping
SCORE: 0.95005 at epoch 28

***** (3/16) *****
Search({'unit_1': 128, 'unit_2': 64, 'opt': 'nadam', 'lr': 0.01, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
SCORE: 0.95073 at epoch 7

***** (4/16) *****
Search({'unit_1': 128, 'unit_2': 64, 'opt': 'nadam', 'lr': 0.001, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00030: early stopping
SCORE: 0.94938 at epoc